In [11]:
from stable_baselines3 import TD3 
import torch
from torch.ao.quantization import quantize_dynamic

h1 = 32
h2 = 32
model = TD3.load(f'../models/TD3_{h1}_{h2}/2500.zip')
policy = model.policy.to(torch.device('cpu'))
policy.eval()
qpolicy = quantize_dynamic(policy, dtype=torch.qint8)
torch.save(qpolicy.state_dict(), f'../models/policies/qpolicy_{h1}_{h2}.pth')
torch.save(policy.state_dict(), f'../models/policies/policy_{h1}_{h2}.pth')

print(qpolicy)

TD3Policy(
  (actor): Actor(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (mu): Sequential(
      (0): DynamicQuantizedLinear(in_features=6, out_features=32, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
      (1): ReLU()
      (2): DynamicQuantizedLinear(in_features=32, out_features=32, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
      (3): ReLU()
      (4): DynamicQuantizedLinear(in_features=32, out_features=2, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
      (5): Tanh()
    )
  )
  (actor_target): Actor(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (mu): Sequential(
      (0): DynamicQuantizedLinear(in_features=6, out_features=32, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
      (1): ReLU()
      (2): DynamicQuantizedLinear(in_features=32, out_features=32, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
      (3): ReLU()
      (4):

In [1]:
from BGN_MC import *
import matplotlib.pyplot as plt
from stable_baselines3 import TD3 

bgn = BGN_MC(tmax=1100, pd=True)
# model = TD3.load('../models/TD3_32/1500.zip', env=bgn)

h1 = 32
h2 = 32

policy_kwargs = dict(activation_fn=torch.nn.ReLU, net_arch=dict(pi=[h1, h2], qf=[h1, h2]))
model = TD3('MlpPolicy', bgn, verbose=1, policy_kwargs=policy_kwargs, learning_rate=0.0001)
qmodel = quantize_dynamic(model.policy.to(torch.device('cpu')), dtype=torch.qint8)
qmodel.load_state_dict(torch.load(f'../models/policies/qpolicy_{h1}_{h2}.pth'))
qmodel.eval()

eis = []
sgis_sums = []
freqs = []
amps = []
r1s = []
for i in range(5):
    observation = bgn.reset()[0]
    terminated = 0
    while terminated != 1:
        # action = model.predict(observation)[0]
        observation = torch.from_numpy(observation).unsqueeze(0).to(torch.device('cpu'))
        action = qmodel(observation).numpy()[0]
        observation, reward, terminated, truncated, info = bgn.step(action)
        sgis = scipy.io.loadmat('bgn_vars.mat')['sgis']
        sgis_sums.append(np.sum(np.mean(np.abs(np.fft.fft(sgis)), axis=0)[1:20]))
        freqs.append(action[0])
        amps.append(action[1])

KeyboardInterrupt: 

In [13]:
print(np.mean(sgis_sums), np.std(sgis_sums))
print(np.mean(freqs), np.mean(amps))


1213.580144690668 286.11659958489287
0.51551336 -0.17702574
